<a href="https://colab.research.google.com/github/12003586/Hackaton/blob/main/Hackaton_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360" -d "/content"

warning [/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #90487).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [8]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360/fruits-360_dataset/fruits-360/Training

[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Kaggle_Fruits360/fruits-360_dataset/fruits-360/Training'
/content


In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import pandas as pd

from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean

from sklearn.model_selection import train_test_split
from skimage import data, color, feature
from skimage.feature import hog

import glob

In [7]:
def load_data(fruit, tipo):
    label=[]
    arr = []
    strr = "Kaggle_Fruits360/"+fruit+"/" + tipo + "/*"
    for file_ in glob.glob(strr):
      img = cv2.imread(file_)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      arr.append(img)
      label.append(fruit)
   
    return arr,label

In [ ]:
def whole_train_data(tipo):
  apples_data, apples_label = load_data('Apples', tipo)
  mangoes_data, mangoes_label = load_data('Mangoes', tipo)
  oranges_data, oranges_label = load_data('Oranges', tipo)
  data =np.concatenate((apples_data,mangoes_data,oranges_data))
  labels =np.concatenate((apples_label, mangoes_label, oranges_label))
  return data, labels

In [ ]:
data_train, labels_train = whole_train_data('Train')
data_test, labels_test = whole_train_data('Test')

In [ ]:
data_train.shape, labels_train.shape

In [ ]:
def preprocessing(arr):
    arr_prep=[]
    for i in range(arr.shape[0]):
        img=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        img=resize(img, (72, 72),anti_aliasing=True)
        arr_prep.append(img)
    return arr_prep

In [ ]:
data_train_p = preprocessing(data_train)
data_test_p = preprocessing(data_test)

In [ ]:
type(data_train[0])

In [ ]:
def ExtractHOG(img):
    ftr,_=hog(img, orientations=8, pixels_per_cell=(16, 16),
            cells_per_block=(1, 1), visualize=True, multichannel=False)
    return ftr
  
def preprocessing_part_two(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(ExtractHOG(arr[i])) 
    return arr_feature

In [ ]:
data_train_ftr = preprocessing_part_two(data_train_p)
data_test_ftr= preprocessing_part_two(data_test_p)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=11)
knn_clf.fit(data_train_ftr, labels_train)

In [ ]:
y_knn_pred = knn_clf.predict(data_test_ftr)

In [ ]:
print(accuracy_score(labels_test, y_knn_pred)*100,'%')

In [ ]:
def showImg(img, name):
    plt.axis("off")
    plt.title(name)
    plt.imshow(img)
    plt.show()

In [ ]:
from random import seed
from random import randint
x_ = randint(0, data_test.shape[0])
showImg(data_test[x_], y_knn_pred[x_])